In [ ]:
# 必要なライブラリのインポート
import os
import json
import getpass
from typing import List # Python標準の型ヒントを使用

# LangChain関連のインポート
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
# from langchain_core.messages import SystemMessage, HumanMessage # ChatPromptTemplateで代替

# --- ユーザー設定項目 ---

# Google Gemini APIキーの入力
api_key_env = os.getenv("GEMINI_API_KEY")
if api_key_env:
    GEMINI_API_KEY = api_key_env
    print("環境変数からGemini APIキーを読み込みました。")
else:
    print("Gemini APIキーを入力してください（入力内容は表示されません）。")
    GEMINI_API_KEY = getpass.getpass()

PROBLEM_TEXT = "XXX"
MAX_DEPTH = 3
INITIAL_VIEWPOINTS = ["AAA", "BBB", "CCC"]
MAX_CHILDREN_PER_NODE = 3
safe_problem_text = PROBLEM_TEXT.replace(' ', '_').replace('/', '_').replace('\\', '_')
OUTPUT_FILENAME = f"{safe_problem_text}_mindmap_gemini.md"
GEMINI_MODEL = "gemini-2.0-flash" # ユーザー指定の gemini-2.0-flash が利用できない場合を考慮し、より一般的な最新モデルに変更
TEMPERATURE = 0.2

print(f"\n--- 設定内容 ---")
print(f"問題の内容: {PROBLEM_TEXT}")
print(f"層数: {MAX_DEPTH}")
print(f"初期観点: {INITIAL_VIEWPOINTS if INITIAL_VIEWPOINTS else 'なし'}")
print(f"各ノードの子要素の最大数: {MAX_CHILDREN_PER_NODE}")
print(f"出力ファイル名: {OUTPUT_FILENAME}")
print(f"使用モデル: {GEMINI_MODEL}") # ユーザー指定が gemini-2.0-flash の場合はここを元に戻してください
print(f"Temperature: {TEMPERATURE}")
print(f"------------------")

環境変数からGemini APIキーを読み込みました。

--- 設定内容 ---
問題の内容: 導電性高分子コンデンサの信頼性悪化
層数: 3
初期観点: ['製造プロセス', '材料', '使用環境']
各ノードの子要素の最大数: 3
出力ファイル名: 導電性高分子コンデンサの信頼性悪化_mindmap_gemini.md
使用モデル: gemini-2.0-flash
Temperature: 0.2
------------------


In [3]:
# LangChain LLM (Gemini) の初期化
llm = None # 初期化失敗に備える
if not GEMINI_API_KEY:
    print("エラー: Gemini APIキーが設定されていません。LLMを初期化できません。")
else:
    try:
        llm = ChatGoogleGenerativeAI(
            model=GEMINI_MODEL,
            google_api_key=GEMINI_API_KEY,
            temperature=TEMPERATURE,
            # GeminiはSystemMessageをネイティブサポートする場合としない場合がある。
            # LangChainが内部でHumanMessageに変換することがある。
            # convert_system_message_to_human=True # 必要に応じて有効化
        )
        # 簡単なテスト呼び出し (APIキーとモデルの有効性を確認)
        # test_response = llm.invoke("こんにちは、テストです。")
        # print(f"LLMテスト応答: {test_response.content[:50]}...") # 応答の先頭を表示
        print(f"LangChain LLM ({GEMINI_MODEL}) が正常に初期化されました。")
    except Exception as e:
        print(f"LLMの初期化に失敗しました: {e}")
        print("考えられる原因: APIキーが無効、指定されたモデル名が利用できない、必要なライブラリが不足しているなど。")
        print(f"指定されたモデル '{GEMINI_MODEL}' が利用可能か確認してください。代替として 'gemini-1.5-flash-latest' や 'gemini-pro' などがあります。")

LangChain LLM (gemini-2.0-flash) が正常に初期化されました。


In [4]:
def generate_factors_from_langchain_gemini(
    parent_factor: str,
    context_problem: str,
    target_depth: int,
    max_depth_of_mindmap: int,
    llm_model, # LangChain llm instance
    viewpoint_context: str = None,
    max_children: int = 3
) -> List[str]:
    """
    LangChain経由でGemini APIを使用し、指定された親要素の下位要因を生成する。

    Args:
        parent_factor: 親要素の名前。
        context_problem: マインドマップ全体の主題。
        target_depth: 生成する要因が属する階層の深さ。
        max_depth_of_mindmap: マインドマップ全体の最大の深さ。
        llm_model: LangChain LLMインスタンス。
        viewpoint_context: 親要素が属する主要な観点。
        max_children: 生成を試みる子要素の最大数。

    Returns:
        生成された要因名 (文字列) のリスト。エラー時や要因が見つからない場合は空リスト。
    """
    if not llm_model:
        print("LLMが初期化されていません。要因生成をスキップします。")
        return []

    # Geminiへの指示。JSON形式での出力を強調。
    json_instruction_prompt = f"""
あなたは、与えられたトピックや要因について、関連性の高い下位要因をマインドマップ形式で提案する専門家アシスタントです。
提案する下位要因は、簡潔で具体的な名称にしてください。
出力は、必ず下記のJSONスキーマに準拠した有効なJSONオブジェクト文字列のみとしてください。説明や前置き、後書きは一切含めないでください。

JSONスキーマ:
{{
  "factors": ["<提案する要因1>", "<提案する要因2>", ..., "<提案する要因{max_children}>"]
}}
"""

    prompt_parts = [f"マインドマップ全体の主題は「{context_problem}」です。"]
    if viewpoint_context and viewpoint_context != parent_factor:
        prompt_parts.append(f"現在フォーカスしている主要な観点は「{viewpoint_context}」です。")
    prompt_parts.append(f"親要素は「{parent_factor}」です。")
    prompt_parts.append(f"この親要素から展開される、第{target_depth}層に該当する具体的な下位要因を{max_children}個程度提案してください。（マインドマップ全体の最大層数は{max_depth_of_mindmap}層です）")
    
    user_request_text = "\n".join(prompt_parts)

    # LangChainのプロンプトテンプレートと出力パーサーを使用
    # SystemMessageはGeminiではHumanMessageに統合されることが多いので、指示はHumanMessageに含める
    # または、ChatPromptTemplateの system メッセージを利用
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", json_instruction_prompt), # GeminiがSystemを解釈する場合
        ("human", user_request_text)
    ])
    
    parser = JsonOutputParser()
    chain = prompt_template | llm_model | parser

    # print(f"\n--- Generating factors for '{parent_factor}' (Target Depth: {target_depth}) using Gemini ---")
    # print(f"Combined Prompt for LLM:\nSystem: {json_instruction_prompt}\nHuman: {user_request_text}\n--------------------")

    try:
        # invokeの入力はテンプレートの変数に合わせる必要があるが、ここでは直接文字列を渡しているので
        # ChatPromptTemplate.from_template を使うか、messages を直接 invoke に渡す
        # ここではテンプレートを使わず直接メッセージリストを作成する方がシンプルかもしれない

        # 修正: ChatPromptTemplateの入力変数を使うか、メッセージリストを直接作成
        # 以下はメッセージリストを直接作成する例
        messages = [
            SystemMessage(content=json_instruction_prompt), # Geminiの挙動によってはHumanに含める
            HumanMessage(content=user_request_text)
        ]
        # response_from_llm = llm_model.invoke(messages) # このままだと Message オブジェクトが返る
        # parsed_result = parser.parse(response_from_llm.content) # そのcontentをパース

        # Chainを使う場合 (ChatPromptTemplate.from_messages で定義したテンプレートと連携)
        # 入力は辞書形式で、テンプレート内のプレースホルダに対応させる
        # この関数ではプレースホルダを使っていないため、invokeに直接メッセージリストを渡すのが適切
        
        # 修正案: invokeに直接メッセージリストを渡す
        full_prompt_messages = [
             # GeminiはSystemプロンプトを特別扱いしないことがあるため、指示を最初のUserターンに含める
             HumanMessage(content=json_instruction_prompt + "\n\n" + user_request_text)
        ]
        
        # もしSystemMessageを使いたい場合
        # full_prompt_messages = [
        #    SystemMessage(content=json_instruction_prompt), # これがHumanに変換されることを期待
        #    HumanMessage(content=user_request_text)
        # ]

        # llm.invoke の結果を直接パースする
        raw_response = llm_model.invoke(full_prompt_messages)
        # print(f"Gemini Raw Response content: {raw_response.content}")
        
        parsed_result = parser.parse(raw_response.content)
        # print(f"Parsed Result: {parsed_result}")

        factors = parsed_result.get("factors", [])
        
        if not isinstance(factors, list) or not all(isinstance(f, str) for f in factors):
            print(f"警告: Geminiからの応答が期待するリスト形式ではありませんでした。 Response: {factors}")
            return []
            
        return factors[:max_children] # 念のため個数制限

    except json.JSONDecodeError as e:
        # parser.parseで失敗した場合もこちらに来る可能性がある
        print(f"エラー: Geminiからの応答のJSONパースに失敗しました。Raw response: '{raw_response.content if 'raw_response' in locals() else 'N/A'}', Error: {e}")
        return []
    except Exception as e:
        # LangChainの呼び出しエラーなども含む
        print(f"エラー: LangChain/Gemini API呼び出し中にエラーが発生しました ({parent_factor}): {e}")
        return []

In [5]:
# この関数は、呼び出す generate_factors_... 関数名と、渡すLLMインスタンスが変わる以外は
# 前回のOpenAI版と同じロジックで動作します。
# 関数名は get_children_recursively_gemini などに変更するとより明確になります。

def get_children_recursively_gemini(
    parent_factor_name: str,
    context_problem: str,
    current_child_depth: int,
    max_mindmap_depth: int,
    llm_instance, # LangChain LLM instance
    viewpoint_context: str = None,
    max_children_to_generate: int = 3
) -> List[dict]:
    """
    指定された親要素の子要素をGemini APIを使って再帰的に生成し、
    子要素のリスト (各要素はノード辞書 {"name": "...", "children": [...]}) を返す。
    """
    if current_child_depth > max_mindmap_depth:
        return []

    # Gemini APIを呼び出して子要素の名前リストを取得
    child_factor_names = generate_factors_from_langchain_gemini( # ここをGemini用の関数に変更
        parent_factor=parent_factor_name,
        context_problem=context_problem,
        target_depth=current_child_depth,
        max_depth_of_mindmap=max_mindmap_depth,
        llm_model=llm_instance, # LLMインスタンスを渡す
        viewpoint_context=viewpoint_context,
        max_children=max_children_to_generate
    )

    if not child_factor_names:
        return []

    children_nodes = []
    indent_prefix = "  " * (current_child_depth -1) 

    for child_name in child_factor_names:
        print(f"{indent_prefix}- 要素 '{child_name}' (第{current_child_depth}層) の下位要素を探索中 (Gemini)...")
        
        grandchildren_nodes = get_children_recursively_gemini( # 再帰呼び出しもこの関数自体
            parent_factor_name=child_name,
            context_problem=context_problem,
            current_child_depth=current_child_depth + 1,
            max_mindmap_depth=max_mindmap_depth,
            llm_instance=llm_instance, # LLMインスタンスを引き継ぐ
            viewpoint_context=viewpoint_context,
            max_children_to_generate=max_children_to_generate
        )
        node = {"name": child_name}
        if grandchildren_nodes:
            node["children"] = grandchildren_nodes
        children_nodes.append(node)
    
    return children_nodes

In [6]:
# このセルは前回のスクリプトから変更ありません。
def convert_node_to_markdown_lines(node, list_depth=0):
    """
    単一のノードとその子をMarkdownのリスト項目に変換する (再帰的)。
    """
    lines = []
    indent = "  " * list_depth
    lines.append(f"{indent}- {node['name']}")
    
    if "children" in node and node["children"]:
        for child_node in node["children"]:
            lines.extend(convert_node_to_markdown_lines(child_node, list_depth + 1))
    return lines

def mindmap_to_markdown_string(mindmap_data):
    """
    完全なマインドマップデータ構造を1つのMarkdown文字列に変換する。
    """
    if not mindmap_data or "name" not in mindmap_data:
        return "# マインドマップの生成に失敗しました"

    markdown_lines = [f"# {mindmap_data['name']}\n"]
    
    if "children" in mindmap_data and mindmap_data["children"]:
        for child_node in mindmap_data["children"]:
            markdown_lines.extend(convert_node_to_markdown_lines(child_node, list_depth=0))
            markdown_lines.append("") 
    
    return "\n".join(markdown_lines)

In [7]:
# LLMクライアントが正常に初期化されている場合のみ実行
if llm: 
    print("\n--- マインドマップ生成開始 (Gemini) ---")
    mindmap_data_structure = {"name": PROBLEM_TEXT, "children": []}

    if MAX_DEPTH < 1:
        print("エラー: 層数は1以上である必要があります。")
    elif MAX_DEPTH == 1:
        print(f"第1層 ({PROBLEM_TEXT}) のみのマインドマップです。")
    else: # MAX_DEPTH >= 2
        if INITIAL_VIEWPOINTS:
            print(f"指定された観点に基づいて第2層を構築します: {INITIAL_VIEWPOINTS}")
            for viewpoint_name in INITIAL_VIEWPOINTS:
                print(f"\n処理中の観点: '{viewpoint_name}' (第2層)")
                children_of_viewpoint = get_children_recursively_gemini( # Gemini版関数を呼び出し
                    parent_factor_name=viewpoint_name,
                    context_problem=PROBLEM_TEXT,
                    current_child_depth=3,
                    max_mindmap_depth=MAX_DEPTH,
                    llm_instance=llm, # 初期化されたLLMインスタンス
                    viewpoint_context=viewpoint_name,
                    max_children_to_generate=MAX_CHILDREN_PER_NODE
                )
                viewpoint_node = {"name": viewpoint_name}
                if children_of_viewpoint:
                    viewpoint_node["children"] = children_of_viewpoint
                mindmap_data_structure["children"].append(viewpoint_node)
        else:
            print("観点の指定がないため、問題の直下の要素 (第2層) を生成します。")
            direct_children = get_children_recursively_gemini( # Gemini版関数を呼び出し
                parent_factor_name=PROBLEM_TEXT,
                context_problem=PROBLEM_TEXT,
                current_child_depth=2,
                max_mindmap_depth=MAX_DEPTH,
                llm_instance=llm, # 初期化されたLLMインスタンス
                viewpoint_context=None,
                max_children_to_generate=MAX_CHILDREN_PER_NODE
            )
            if direct_children:
                mindmap_data_structure["children"] = direct_children

    print("\n--- マインドマップ生成完了 (Gemini) ---")

    markdown_output_string = mindmap_to_markdown_string(mindmap_data_structure)

    try:
        with open(OUTPUT_FILENAME, "w", encoding="utf-8") as f:
            f.write(markdown_output_string)
        print(f"\nマインドマップがMarkdownファイル '{OUTPUT_FILENAME}' に保存されました。")
    except IOError as e:
        print(f"エラー: ファイル '{OUTPUT_FILENAME}' の書き込みに失敗しました: {e}")
    except Exception as e:
        print(f"予期せぬエラーが発生しました: {e}")

    print("\n--- 生成されたMarkdown (プレビュー) ---")
    print(markdown_output_string)
    print("------------------------------------")

else:
    print("LangChain LLM (Gemini) が初期化されていないため、マインドマップ生成処理をスキップしました。")
    print("セル1およびセル2の設定とAPIキーを確認してください。")


--- マインドマップ生成開始 (Gemini) ---
指定された観点に基づいて第2層を構築します: ['製造プロセス', '材料', '使用環境']

処理中の観点: '製造プロセス' (第2層)
エラー: LangChain/Gemini API呼び出し中にエラーが発生しました (製造プロセス): name 'SystemMessage' is not defined

処理中の観点: '材料' (第2層)
エラー: LangChain/Gemini API呼び出し中にエラーが発生しました (材料): name 'SystemMessage' is not defined

処理中の観点: '使用環境' (第2層)
エラー: LangChain/Gemini API呼び出し中にエラーが発生しました (使用環境): name 'SystemMessage' is not defined

--- マインドマップ生成完了 (Gemini) ---

マインドマップがMarkdownファイル '導電性高分子コンデンサの信頼性悪化_mindmap_gemini.md' に保存されました。

--- 生成されたMarkdown (プレビュー) ---
# 導電性高分子コンデンサの信頼性悪化

- 製造プロセス

- 材料

- 使用環境

------------------------------------
